In [3]:
from google.colab import userdata
import os

os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

In [4]:
!pip install -q youtube-transcript-api langchain-community langchain-HuggingFace \
               faiss-cpu tiktoken python-dotenv

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import ChatHuggingFace, HuggingFaceEmbeddings,HuggingFaceEndpoint
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

## S-1 Indexing


###1a loader

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled

video_id = "8IU7YBgpQxg"

try:
    fetched_transcript = YouTubeTranscriptApi().fetch(video_id, languages=['en'])
    transcript_list = fetched_transcript.to_raw_data()
    transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript)

except TranscriptsDisabled:
    print("No captions available for this video.")
except Exception as e:
    print(f"An error occurred: {e}")


in your eyes where is this warlike situation Israel Iran going to some extent it's a religious issue it's about Muslims of Jews and they are trying to create this pan Turan Corridor which will essentially Wipe Out the nation of Armenia and create a whole turkey Corridor this Iran thing came up the Gaza Israel thing is already going on what do you think the mood is like in that Nation Gaza has been sealed off Palestinians obviously want Israel to cease to exist but long term there is this deep hatred which is like a blood hatred what's been India's stand on both these conflicts India's borders are likely to change in some way in which direction I would not like to talk about it he said it already what's happening on a global scale Europe is in recession people are worried in Europe that Putin may have expansionist Ambitions and would want to take Poland Hungary there's a cold war happening in Africa as well drone Warfare is now an out andout reality it's gone from being something theore

### 1b splitter

In [ ]:
splitter= RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks= splitter.create_documents([transcript])

In [ ]:
len(chunks)

88

### s- 1c & 1d (embedding & storinging Vec Store)

In [ ]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
vector_store= FAISS.from_documents(chunks,embeddings)

## S-2 Retrival

In [ ]:
retriver= vector_store.as_retriever(search_type='mmr',search_kwargs={"k":6, "fetch_k": 20})

In [ ]:
retriver

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7e55b4e1b980>, search_type='mmr', search_kwargs={'k': 6, 'fetch_k': 20})

In [ ]:
retriver.invoke('What is deepmind')

[Document(id='2b7ab035-d267-488f-8f43-03b756135b25', metadata={}, page_content="these little weapons these drones the these lering drones and even anti-personnel drones which take out one person or two people it's like a f flying hand grenade it will chase a soldier around and once it is like near enough it will explode it will delate and it's just a small determination like a single hand grenade but it can kill or MIM three or four people at at a given time so we are seeing this very new kind of warfare autonomous weapon systems that think on their own that don't always need inputs from the remote controlling person far away and they can take decisions on their own we are beginning to see the the the influence of AI in this you know very very rudimentary AI systems B put on a chip on a drone and this drone can go and kill um whoever it wants to kill on its own sometimes so we are seeing this happening a whole new level of warfare is coming up uh tanks could become obsolete if sufficie

## S-3 Agumentation

In [ ]:
endpoint = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.2",
    task="text-generation",
    temperature=0.2,
)

#  Wrap as CHAT model
llm = ChatHuggingFace(llm=endpoint)

In [ ]:
prompt = PromptTemplate(
    template="""
You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}
    """,
    input_variables=['context', 'question']
)


In [ ]:
question          = input("enter question")
retrived_docs    = retriver.invoke(question)

enter questionis nuclear weapon discussed in the video and explain various nuclear weapons in points


In [ ]:
retrived_docs

[Document(id='2b7ab035-d267-488f-8f43-03b756135b25', metadata={}, page_content="these little weapons these drones the these lering drones and even anti-personnel drones which take out one person or two people it's like a f flying hand grenade it will chase a soldier around and once it is like near enough it will explode it will delate and it's just a small determination like a single hand grenade but it can kill or MIM three or four people at at a given time so we are seeing this very new kind of warfare autonomous weapon systems that think on their own that don't always need inputs from the remote controlling person far away and they can take decisions on their own we are beginning to see the the the influence of AI in this you know very very rudimentary AI systems B put on a chip on a drone and this drone can go and kill um whoever it wants to kill on its own sometimes so we are seeing this happening a whole new level of warfare is coming up uh tanks could become obsolete if sufficie

In [ ]:
context_text = "\n\n".join(doc.page_content for doc in retrived_docs)
context_text

"have sufficient facile material nuclear facile material to perhaps construct a few cudim nuclear bombs and that is something that worries Israel a lot because Israel also is a nuclear weapons power and unstated nuclear weapons power but it's believed to have about let's say 90 or 100 nuclear warheads so there is all these Dynamics happening in the region there's also the Turkish angle north of Iran you have the nation of aeran which is a very strong Turkish Ally or proxy and they are trying to create this pan Turan Corridor which will essentially Wipe Out the nation of Armenia and create a whole turkey corridor from Turkey in the west all the way to turkistan and other turky nations in the East Central Asia region which is something that Iran would not like to see happen even the Americans would not like to see this happen even the Russians don't want this so there's a whole lot going on the Middle East is always a very complicated region but you have that and then what happens on\n\n

In [ ]:
final_prompt=prompt.invoke({"context":context_text,"question":question})

## 4- Generation

In [ ]:
answer = llm.invoke(final_prompt)
print(answer.content)

 Yes, nuclear weapons are discussed in the video. The context mentions that Iran is believed to have around 90 or 100 nuclear warheads, making it a nuclear weapons power. The video also mentions Russia, which is believed to have enough nuclear weapons to annihilate the world if they choose to use them. China is also mentioned as the second most powerful nation but having fewer nuclear weapons than Russia. The video also touches upon the influence of AI in warfare, mentioning autonomous weapon systems and drones that can make decisions and take actions on their own. However, the video does not go into detail about the types or capabilities of specific nuclear weapons.


## Building chain

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [ ]:
def format_docs(retrieved_docs):
  context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
  return context_text

In [ ]:
parallel_chain = RunnableParallel({
    'context': retriver | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [ ]:
parallel_chain.invoke('who is Demis')

{'context': "between Israel and the nation of Jordan from there to the militarian Sea you have the small nation of Israel from The River To The Sea if Palestine is free it means there is no Israel left so that is obviously a call for the uh Erasure of a nation and the genocide of a people now obviously that is something that no civilized person would want anywhere in the world so that's not a viable solution but then the Palestinians said that the Israelis are oppressing us well the Israelis gave them a separate nation essentially in Gaza and the Israelis were no longer interfering in what was happening in Gaza but the Palestinians were not okay with that so I don't know the Palestinians would like the complete destruction of Israel the Israelis just want to they also I me they are not entirely All Saints themselves there is the West Bank region which in which they have been settling people maybe they should not do that certainly they should not do that and so on so both sides have don

In [ ]:
parser = StrOutputParser()

In [ ]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke('Can you sumerize the video')

" The video discusses the use of drones in warfare, specifically in the Second Nagorno-Karabakh Conflict in 2020, and the Chavda Index of hard power. The speaker mentions the first use of loitering munitions or drones in the conflict, which can fly for hours and then attack, taking out tanks and causing significant damage. The speaker also talks about constructing a hard power score for each nation based on various factors, with the US being the most powerful, Russia as the near superpower, China as a great power, and India as a regional power. The speaker then discusses the complicated issues in the Middle East, specifically Gaza and the West Bank, and the potential for continued tension and conflict. The speaker also mentions Putin's expansionist ambitions and the ongoing cold war in Africa, as well as the increasing reality of drone warfare. Abhijit Chav, a guest on the podcast, is welcomed back and they discuss the geopolitical updates of the year, including elections in important 